In [ ]:
# 본인이 현재 사용중인 통신사 사이트 로그인 후 최근 요금명세서 주요항목을 수집하세요.
# 교보문고에 로그인하여 주문 배송조회 페이지의 도서명을 수집하세요.
# ChatGPT 로그인 후 히스토리에서 최근 5개 질문을 수집하세요.
# Github 로그인 후 내 레포지토리 리스트를 수집하세요.
# 슬랙에 로그인 후 특정 채널의 최근 메시지 50개를 수집하세요.

In [ ]:
# 본인이 현재 사용중인 통신사 사이트 로그인 후 최근 요금명세서 주요항목을 수집하세요.

In [1]:
from selenium.webdriver.common.keys import Keys 

def get_product_links(search_url, max_links):
    """스크롤을 반복하며 상품 상세 페이지 링크를 수집하고 진행 상황을 출력합니다."""
    driver.get(search_url)
    product_links = set()
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_count = 0
    
    # ⭐ 시작 알림
    print("\n==============================================")
    print(f"🔗 1단계: 상품 링크 수집 시작 ({driver.current_url})")
    print(f"🔗 목표 링크 수: {max_links}개")
    print("==============================================")

    while len(product_links) < max_links:
        # 1. 페이지 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3) # 데이터 로드 대기 시간
        scroll_count += 1
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # 2. 종료 조건 확인
        if new_height == last_height and scroll_count > 1:
            print(f"🛑 스크롤 멈춤. 검색 결과의 끝에 도달했습니다. (스크롤 횟수: {scroll_count})")
            break
            
        last_height = new_height
        
        # 3. 현재까지 로드된 모든 상품 링크 수집
        try:
            items = driver.find_elements(By.CSS_SELECTOR, 'a[data-product-id]')
            new_links_found = 0
            
            for item in items:
                link = item.get_attribute('href')
                if link and link.startswith(BASE_URL + '/vp/products/'):
                    clean_url = link.split('?')[0]
                    if clean_url not in product_links:
                        product_links.add(clean_url)
                        new_links_found += 1
            
            # ⭐ 진행 상황 출력
            print(f"✅ 스크롤 {scroll_count:02d}회 | 신규 링크 {new_links_found}개 발견 | 총 링크 수: {len(product_links)} / {max_links}")
            
            if len(product_links) >= max_links:
                break
                
        except Exception as e:
            print(f"⚠️ 링크 수집 중 오류 발생: {e}")
            break
            
    return list(product_links)[:max_links]

In [2]:
def extract_product_details(product_url, index, total):
    """개별 상세 페이지를 방문하여 데이터를 추출하고 진행 상황을 출력합니다."""
    
    # ⭐ 진행 상황 출력
    print(f"\n➡️ [{index:03d}/{total}] 방문 URL: {product_url}")
    
    driver.get(product_url)
    
    try:
        # (로딩 대기 코드는 이전과 동일)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.prod-buy-header'))
        )
        
        # --- 데이터 추출 (이전과 동일) ---
        # (브랜드, 상품명, 가격 등을 추출하는 코드)
        
        # 상품명 추출은 반드시 성공해야 함
        try:
            product_name = driver.find_element(By.CSS_SELECTOR, '.twc-font-bold').text.strip()
        except:
            product_name = 'N/A' 

        # (나머지 가격, 리뷰 수 등 데이터 추출)

        data = {
            'URL': product_url,
            '브랜드': brand, # 추출된 변수 사용
            '상품명': product_name, 
            # ... 나머지 데이터 필드 ...
        }
        
        # ⭐ 성공 시 상품명 출력
        print(f"✨ 성공! 크롤링 상품: {data['상품명'][:50]}...")
        return data

    except TimeoutException:
        print(f"❌ 실패: 로딩 시간 초과.")
        return None
    except Exception as e:
        print(f"⚠️ 실패: 데이터 추출 오류. ({e})")
        return None

In [3]:
def main():
    try:
        # (드라이버 초기화 코드는 이전과 동일)

        # 1. 링크 수집
        product_links = get_product_links(SEARCH_URL, MAX_ITEMS)
        total_links = len(product_links)
        print(f"\n==============================================")
        print(f"✅ 1단계 완료: 총 {total_links}개의 링크 수집 완료")
        print(f"🔗 2단계: 상세 정보 추출 시작")
        print("==============================================")
        
        # 2. 상세 정보 추출
        for i, link in enumerate(product_links):
            # i+1 = 현재 순서, total_links = 총 개수
            product_data = extract_product_details(link, i + 1, total_links) 
            if product_data:
                all_products_data.append(product_data)
            
            # 지연 시간
            time.sleep(3) 

        # 3. CSV 파일로 저장 (이전 코드와 동일)

    finally:
        driver.quit() 
        print("\n브라우저 드라이버를 종료합니다.")

In [5]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def extract_product_details(product_url, index, total):
    """개별 상세 페이지를 방문하여 데이터를 추출하고 진행 상황을 출력합니다."""
    
    # ... (생략: URL 방문 및 대기 코드) ...
    
    driver.get(product_url)
    
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.prod-buy-header'))
        )
        
        # --- 데이터 추출 (기존 필드) ---
        
        # ... (생략: 브랜드, 상품명, 가격, 리뷰 수 추출 코드는 이전과 동일) ...
        # (편의상 N/A 처리 코드는 생략하고 핵심 로직만 표시합니다)
        
        # 상품명 추출
        product_name = driver.find_element(By.CSS_SELECTOR, '.twc-font-bold').text.strip()
        
        # 1정당 가격 추출
        price_per_unit_text = driver.find_element(By.CSS_SELECTOR, '.custom-oos.fw-text-\\[12px\\]\\/\\[15px\\]\\.fw-text-bluegray-900').text.strip()
        
        # 리뷰 수 추출
        review_count_text = driver.find_element(By.CSS_SELECTOR, '.rating-count-txt').text.strip()
        
        
        # --- ⭐️ 별점 (Rating) 추출 로직 추가 ---
        rating = 'N/A'
        
        # 1. 시각적인 막대 너비(Width)를 이용해 추출 시도
        try:
            # 별점 막대 요소 선택
            rating_bar = driver.find_element(By.CSS_SELECTOR, '.rating-star-num')
            
            # width 스타일 값을 가져옴 (예: 'width: 90%;')
            style_attr = rating_bar.get_attribute('style')
            
            # style 문자열에서 width 값을 파싱
            if style_attr:
                width_percent_match = [s for s in style_attr.split(';') if 'width' in s]
                if width_percent_match:
                    width_percent_str = width_percent_match[0].split(':')[-1].strip().replace('%', '')
                    
                    try:
                        width_percent = float(width_percent_str)
                        # 너비 (0-100)를 별점 (0.0-5.0)으로 변환
                        rating_value = round((width_percent / 100.0) * 5.0, 2)
                        rating = str(rating_value)
                    except ValueError:
                        pass # 숫자로 변환 실패 시 무시
        except NoSuchElementException:
             # 요소가 없거나 실패한 경우, 두 번째 방법 시도
            pass

        # 2. (추가) 별점 텍스트가 다른 숨겨진 요소에 있을 경우 (Coupang의 일반적인 방식)
        if rating == 'N/A'

SyntaxError: expected ':' (394445353.py, line 61)

In [ ]:
# main 함수 내 CSV 저장 부분

if all_products_data:
    # 필드 순서 조정 (필요하다면)
    keys = ['URL', '브랜드', '상품명', '할인전 가격', '가격', '1정당 가격', '별점', '리뷰수'] 
    filename = "coupang_selenium_data.csv"
    
    with open(filename, 'w', newline='', encoding='utf-8-sig') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(all_products_data)
    
    print(f"\n🎉 크롤링 완료! {len(all_products_data)}개 데이터를 {filename}에 저장했습니다.")

In [7]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import urllib.parse
import pandas as pd
import re

# --------------------------------------------------
# 1) 드라이버 설정
# --------------------------------------------------
service = Service(ChromeDriverManager().install())
options = Options()
#options.add_argument("--headless=new")          # UI 없는 최신 headless
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")
options.add_argument("--lang=ko_KR")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument(
    "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36"
)

driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 12)

# --------------------------------------------------
# 2) 파라미터
# --------------------------------------------------
query = "영양제" # 비티민 필요?
encoded_query = urllib.parse.quote(query)
max_pages = 3               # 수집할 검색 페이지 수
DO_DETAIL = True            # 상세 페이지 진입해서 브랜드/1정당가격 시도

results = []

# --------------------------------------------------
# 3) 페이지 루프
#   - 쿠팡 검색 결과의 각 카드(li.search-product)에서 필드 추출
# --------------------------------------------------
for page in range(1, max_pages + 1):
    url = f"https://www.coupang.com/np/search?q={encoded_query}&channel=user&page={page}"
    driver.get(url)
    # 첫 로딩 대기
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.search-product")))
    time.sleep(1.2)  # 살짝 추가 대기 (이미지/가격 lazy load 보정)

    items = driver.find_elements(By.CSS_SELECTOR, "li.search-product")
    for it in items:
        # 광고/품절 카드 스킵(필요 시)
        try:
            if it.get_attribute("class") and "advertising" in it.get_attribute("class"):
                continue
        except:
            pass

        # 필드 추출(검색 결과에서 바로 뽑기)
        try:
            link_el = it.find_element(By.CSS_SELECTOR, "a.search-product-link")
            product_url = link_el.getAttribute("href") if hasattr(link_el, "getAttribute") else link_el.get_attribute("href")
        except:
            product_url = ""

        try:
            product_name = it.find_element(By.CSS_SELECTOR, "div.name").text.strip()
        except:
            product_name = ""

        # 가격(최종가 / 정가)
        try:
            price = it.find_element(By.CSS_SELECTOR, "strong.price-value").text.strip()
        except:
            price = ""

        # 정가는 경우에 따라 없을 수 있음
        original_price = ""
        for sel in ["del.price-old", "span.price > del", "div.price-amount.original-price-amount"]:
            try:
                original_price = it.find_element(By.CSS_SELECTOR, sel).text.strip()
                if original_price:
                    break
            except:
                continue

        # 별점/리뷰수
        try:
            rating = it.find_element(By.CSS_SELECTOR, "em.rating").text.strip()
        except:
            rating = ""

        try:
            review_txt = it.find_element(By.CSS_SELECTOR, "span.rating-total-count").text.strip()  # "(1,234)"
            review_count = re.sub(r"[^\d]", "", review_txt)  # 숫자만 추출
        except:
            review_count = ""

        # 썸네일
        try:
            img_url = it.find_element(By.CSS_SELECTOR, "img.search-product-wrap-img").get_attribute("src")
        except:
            img_url = ""

        # 기본 레코드
        rec = {
            "브랜드": "",
            "상품명": product_name,
            "가격(최종)": price,
            "정가": original_price,
            "별점": rating,
            "리뷰수": review_count,
            "상품URL": product_url,
            "이미지": img_url,
            "1정당가격": ""
        }

        # --------------------------------------------------
        # 4) 상세 페이지에서 추가 정보 시도 (선택)
        #   - 브랜드, 1정당 가격(있을 때만)
        #   - 쿠팡은 페이지 타입이 다양하므로 여러 셀렉터를 시도
        # --------------------------------------------------
        if DO_DETAIL and product_url:
            try:
                driver.execute_script("window.open('','_blank');")
                driver.switch_to.window(driver.window_handles[-1])
                driver.get(product_url)
                # 상세 로딩 대기
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "body")))
                time.sleep(1.2)

                brand = ""
                # 브랜드 셀렉터 후보
                for sel in [
                    "a.prod-brand-name",                  # 구형
                    ".prod-buy-header__brand a",          # 신형
                    "div.prod-brand-name a",
                    "span.prod-brand-name a"
                ]:
                    try:
                        brand = driver.find_element(By.CSS_SELECTOR, sel).text.strip()
                        if brand:
                            break
                    except:
                        continue

                # 1정당 가격 텍스트 후보(페이지에 있을 때만)
                per_unit = ""
                try:
                    # 예) "(1정당 70원)" 같은 텍스트
                    per_unit_el = driver.find_element(By.XPATH, "//*[contains(text(),'정당') or contains(text(),'1정당')]")
                    per_unit = per_unit_el.text.strip()
                except:
                    per_unit = ""

                rec["브랜드"] = brand
                rec["1정당가격"] = per_unit

            except Exception:
                pass
            finally:
                # 상세 탭 닫고 검색 결과 탭으로 복귀
                if len(driver.window_handles) > 1:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

        results.append(rec)

    print(f"[INFO] page {page} done. collected: {len(results)}")
    time.sleep(1.0)  # 페이지 간 간격

driver.quit()

# --------------------------------------------------
# 5) DataFrame 및 엑셀 저장
# --------------------------------------------------
df = pd.DataFrame(results)
# 정리: 가격 문자열 → 숫자(옵션)
for col in ["가격(최종)", "정가"]:
    if col in df.columns:
        df[col] = df[col].str.replace(",", "").str.replace("원", "").str.strip()
        df[col] = pd.to_numeric(df[col], errors="coerce")

# 저장
out_name = f"쿠팡_{query}_크롤링.xlsx"
df.to_excel(out_name, index=False)
print("엑셀 저장 완료 ->", out_name)


TimeoutException: Message: 


In [ ]:
import requests
import time
import urllib.parse
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup

BASE = "https://www.coupang.com"
SEARCH_PATH = "/np/search"

# 1) robots.txt 검사 (먼저)
rp = RobotFileParser()
rp.set_url(BASE + "/robots.txt")
rp.read()

query = "영양제"
params = {"q": query, "channel": "recent"}
url = BASE + SEARCH_PATH + "?" + urllib.parse.urlencode(params)

# robots.txt에서 허용되는지 확인
if not rp.can_fetch("*", url):
    raise SystemExit("robots.txt에서 크롤링 금지된 경로입니다. 허가를 받거나 다른 방법을 사용하세요.")

# 2) 요청과 재시도(정중한 간격)
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9"
}

def fetch_with_retries(url, max_retries=3, backoff=1.0):
    for i in range(max_retries):
        resp = requests.get(url, headers=HEADERS, timeout=10)
        if resp.status_code == 200:
            return resp
        # 서버가 429/503 등 반환하면 재시도 전 지수적 대기
        time.sleep(backoff * (2 ** i))
    resp.raise_for_status()

resp = fetch_with_retries(url)
soup = BeautifulSoup(resp.text, "html.parser")

# 3) 필요한 데이터 추출(예: 검색 결과 카드에서 상품명/가격 등)
items = soup.select("li.search-product")
results = []
for it in items:
    name = it.select_one("div.name")
    price = it.select_one("strong.price-value")
    link = it.select_one("a.search-product-link")
    results.append({
        "상품명": name.get_text(strip=True) if name else "",
        "가격": price.get_text(strip=True) if price else "",
        "링크": urllib.parse.urljoin(BASE, link['href']) if link else ""
    })
    # 아주 중요: 요청 사이에 충분한 지연 두기 (서버 부담 완화)
    time.sleep(0.3)

# 4) 결과 저장
import pandas as pd
df = pd.DataFrame(results)
df.to_excel("쿠팡_결과.xlsx", index=False)
